In [2]:
import numpy as np
import math
import scipy.stats
import random


sample = [[83, 85], 
          [84, 85, 85, 86, 87], 
          [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90],
          [89, 90, 90, 91],
          [90, 92]]

# a) Определить влияние возраста на содержание иммуногобулина в крови с помощью регрессионного анализа

In [9]:
def TSS(y):
    y_average = sum(y)/len(y)
    res = 0
    for i in range(len(y)):
        res += (y[i] - y_average)**2
    return res

def RSS(e):
    return np.matmul(e.transpose(), e)

def R_2(tss, rss):
    return (tss - rss)/tss

def calc_koefs(ksi, y):
    ksi_T = ksi.transpose()
    F = np.matmul(ksi_T, ksi)
    F_inv = np.linalg.inv(F)
    koefs = np.matmul(F_inv, np.matmul(ksi_T, y))
    return koefs

def calc_e(ksi, y, koefs):
    return y - np.matmul(ksi, koefs)




ksi = []
y = []
for i in range(len(sample)):
    for j in range(len(sample[i])):
        line = [0]*len(sample)
        line[i] = 1
        ksi.append(line)
        y.append(sample[i][j])
n = len(ksi)
ksi = np.array(ksi)
y = np.array(y)

koefs = calc_koefs(ksi, y)

print('Коэффициенты: \n')
for i in range(len(koefs)):
    print(f'b{i} = {koefs[i]}')

Коэффициенты: 

b0 = 84.0
b1 = 85.4
b2 = 87.81818181818183
b3 = 90.0
b4 = 91.0


In [14]:
alpha = 0.05

def calc_delta(ksi, koefs, i, rss):
    ksi_T = ksi.transpose()
    F = np.matmul(ksi_T, ksi)
    F_inv = np.linalg.inv(F)
    p = ksi.shape[1]
    return koefs[i]*math.sqrt(n-p)/math.sqrt(rss*F_inv[i][i])

def integrate_student(a, b, N):
    def student(x):
        return math.gamma((N+1)/2)/(math.sqrt(math.pi*N)*math.gamma(N/2)*(1+x**2/N)**((N+1)/2))
    return scipy.integrate.quad(student, a, b)[0]
    
e = calc_e(ksi, y, koefs)
rss = RSS(e)
    
deltas = [calc_delta(ksi, koefs, i, rss) for i in range(len(sample))]

p_values = [2*integrate_student(abs(deltas[i]), math.inf, n) for i in range(len(deltas))]
for i in range(len(sample)):
    if p_values[i] < alpha:
        print(f'b{i} значим')
    else:
        print(f'b{i} НЕ значим')

b0 значим
b1 значим
b2 значим
b3 значим
b4 значим


In [15]:
def integrate_fisher(a, b, d1, d2):
    def func(x):
        return x**(d1/2-1)*(1-x)**(d2/2-1)
    Beta = scipy.integrate.quad(func, 0, 1)[0]
    def fisher(x):
        return pow(d1/d2, d1/2) * pow(x, (d1/2 - 1)) * pow((1 + d1*x/d2), -(d1+d2)/2) / Beta 
    return scipy.integrate.quad(fisher, a, b)[0]


tss = TSS(y)
r_2 = R_2(tss, rss)
print('Коэффициент детерминации: R^2 =', r_2)

p = ksi.shape[1]
delta_r_2 = r_2*(n-p)/((1-r_2)*(p-1))

p_value_r_2 = integrate_fisher(delta_r_2, math.inf, p-1, n-p)
print('p-value =', p_value_r_2)
if p_value_r_2 < alpha:
    print('Коэффициент детерминации значим')
else:
    print('Коэффициент детерминации НЕ значим')

Коэффициент детерминации: R^2 = 0.8091004084734492
p-value = 1.2778798793544494e-06
Коэффициент детерминации значим


# b) Провести попарное сравнение средних в рамках регрессионной модели

In [20]:
ksi_T = ksi.transpose()
F = np.matmul(ksi_T, ksi)
F_inv = np.linalg.inv(F)
    

for i in range(p):
    for j in range(i+1, p):
        delta_comp = (koefs[i]-koefs[j])*math.sqrt(n-p)/math.sqrt(rss*(F_inv[i][i]+F_inv[j][j]))
        p_value = 2*integrate_student(abs(delta_comp), math.inf, n-p)
        print(f'H0: b{i}=b{j} | H1: !H0     ->     p-value = {p_value} =>', end=' ')
        if p_value < alpha:
            print('Отвергаем H0')
        else:
            print('Не можем отвергнуть H0')
        

H0: b0=b1 | H1: !H0     ->     p-value = 0.14340811626585315 => Не можем отвергнуть H0
H0: b0=b2 | H1: !H0     ->     p-value = 0.0002287219868382402 => Отвергаем H0
H0: b0=b3 | H1: !H0     ->     p-value = 4.586693462926683e-06 => Отвергаем H0
H0: b0=b4 | H1: !H0     ->     p-value = 4.003248689410462e-06 => Отвергаем H0
H0: b1=b2 | H1: !H0     ->     p-value = 0.0006244964048529733 => Отвергаем H0
H0: b1=b3 | H1: !H0     ->     p-value = 5.249414227098847e-06 => Отвергаем H0
H0: b1=b4 | H1: !H0     ->     p-value = 7.186665224756283e-06 => Отвергаем H0
H0: b2=b3 | H1: !H0     ->     p-value = 0.0029472837797510382 => Отвергаем H0
H0: b2=b4 | H1: !H0     ->     p-value = 0.0012792424725151822 => Отвергаем H0
H0: b3=b4 | H1: !H0     ->     p-value = 0.3054360645262756 => Не можем отвергнуть H0
